In [1]:
import numpy as np

In [2]:
import pandas as pd
import seaborn as sns

In [3]:
from scipy import stats

In [4]:
import matplotlib.pyplot as plt

In [5]:
import statsmodels.api as sm

In [9]:
from statsmodels.stats import diagnostic as diag
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
%matplotlib inline

//LOAD THE DATA TO SFRAME

In [10]:
path=r"C:\Users\OEM\Desktop\Datascience details\Regression\myCode\InputFile\ipl\matches.csv"


In [11]:
IPL_Match_df=pd.read_csv(path)

In [12]:
#select first 11 columns from dataset in which we are going to work
IPL_Match_df = IPL_Match_df.iloc[:,0:11]
IPL_Match_df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore


//CLEAN THE DATA

In [13]:
IPL_Match_df.dtypes

id                int64
season            int64
city             object
date             object
team1            object
team2            object
toss_winner      object
toss_decision    object
result           object
dl_applied        int64
winner           object
dtype: object

In [14]:
#check for null values
display(IPL_Match_df.isnull().any())

id               False
season           False
city              True
date             False
team1            False
team2            False
toss_winner      False
toss_decision    False
result           False
dl_applied       False
winner            True
dtype: bool

In [15]:
IPL_Match_df=IPL_Match_df.dropna()

In [16]:
display(IPL_Match_df.isnull().any())

id               False
season           False
city             False
date             False
team1            False
team2            False
toss_winner      False
toss_decision    False
result           False
dl_applied       False
winner           False
dtype: bool

In [70]:
#loading dependent and independant variables
X=IPL_Match_df.drop(['id','season','date','result','dl_applied','winner'],axis=1)
Y=IPL_Match_df.drop(['id','season','date','team1','team2','result','dl_applied','city','toss_winner','toss_decision'],axis=1)

In [71]:
pd.set_option('display.max_columns', None)
X.head()


,city,team1,team2,toss_winner,toss_decision
0,Hyderabad,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field
1,Pune,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field
2,Rajkot,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field
3,Indore,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field
4,Bangalore,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat


In [19]:
Y.head()

,winner
0,Sunrisers Hyderabad
1,Rising Pune Supergiant
2,Kolkata Knight Riders
3,Kings XI Punjab
4,Royal Challengers Bangalore


CONVERTING VARIABLE COLUMNS TO NUMERIC

In [72]:
#Updating city column
city=pd.get_dummies(X['city'],drop_first='true')
X=X.drop('city',axis=1)
X=pd.concat([X,city],axis=1)


In [73]:
#Updating toss_winner column
toss_winner=pd.get_dummies(X['toss_winner'],drop_first='true')
X=X.drop('toss_winner',axis=1)
X=pd.concat([X,toss_winner],axis=1)

In [74]:
#Updating toss_decision column
toss_decision=pd.get_dummies(X['toss_decision'],drop_first='true')
X=X.drop('toss_decision',axis=1)
X=pd.concat([X,toss_decision],axis=1)

In [75]:
#Updating team1 and team2 column
team1=pd.get_dummies(X['team1'],drop_first='true')
X=X.drop('team1',axis=1)
X=pd.concat([X,team1],axis=1)
team2=pd.get_dummies(X['team2'],drop_first='true')
X=X.drop('team2',axis=1)
X=pd.concat([X,team2],axis=1)

In [76]:
X.head()

,Ahmedabad,Bangalore,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,field,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [25]:
Y.head()

,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad
0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0


In [30]:
#split the data to training and test
X_train,X_test,Y_train,Y_test=train_test_split( X, Y, test_size=0.2, random_state=1)

In [37]:
regression_model = LinearRegression()

In [62]:
regression_model.fit()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [63]:
Y_predict = regression_model.predict(X_test)

In [73]:
X_test[:5]


,Ahmedabad,Bangalore,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,...,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,field
562,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [79]:
pd.set_option('display.max_columns', 500)
Y_test[:5]

,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad
562,0,0,0,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,1,0,0,0,0,0,0
41,0,0,0,1,0,0,0,0,0,0,0,0,0
609,0,0,0,0,0,0,0,0,0,0,0,0,1
372,1,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
Y_predict[

array([[ 2.03783868e-02,  4.00516200e-02, -7.28518427e-04,
         2.47888508e-02, -4.68347858e-03, -1.92560194e-02,
         3.06479415e-02,  1.82376627e-02, -2.72077438e-02,
        -7.04885602e-03,  1.96801053e-03,  4.31285807e-02,
        -1.56905743e-02],
       [-1.00074746e-02,  3.02468514e-01,  2.26660433e-03,
         4.24439448e-02,  3.18726100e-03,  3.60454366e-02,
         4.63426905e-01,  3.40245799e-02,  5.23587366e-02,
         2.59326363e-02, -3.99048425e-03,  4.94441325e-03,
         5.73761087e-03],
       [ 4.08919956e-02,  1.30801052e-02,  4.28469472e-03,
         5.50721910e-02, -3.87410967e-03,  7.12078001e-02,
         7.07304244e-02, -1.86092334e-02, -1.92534612e-02,
         4.06404894e-02, -1.42883653e-03,  7.00351084e-01,
         2.00850930e-02],
       [-2.58778031e-03, -3.75593152e-02,  2.47080984e-04,
         7.08779303e-02,  6.74135759e-03,  5.74506140e-02,
         9.31645858e-02, -1.22146671e-03,  5.72328655e-02,
         5.85806105e-02, -7.62552898e

In [74]:
score=r2_score(Y_test,Y_predict)


In [76]:
print (score)

-0.19581094629026952


In [ ]:
##Accuracy is negaive as we have used regression model for a classification problem
##Lets try with the classification model as below

In [ ]:
sns.pairplot(X,hue='winner')

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
classification_model = LogisticRegression()

In [36]:
X_train.head()

,Ahmedabad,Bangalore,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,field
347,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
631,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
555,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [41]:
Y_train

,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad
347,1,0,0,0,0,0,0,0,0,0,0,0,0
631,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0
555,0,0,0,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,0,0,0
330,0,1,0,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,1,0,0,0,0
260,0,0,0,0,0,0,0,0,1,0,0,0,0
186,0,0,0,0,0,0,0,0,1,0,0,0,0
515,0,0,0,1,0,0,0,0,0,0,0,0,0


In [43]:
IPL_Match_df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore


In [55]:
Output_df = IPL_Match_df.iloc[:,[10]]

In [56]:
Output_df.head()

,winner
0,Sunrisers Hyderabad
1,Rising Pune Supergiant
2,Kolkata Knight Riders
3,Kings XI Punjab
4,Royal Challengers Bangalore


In [57]:
X.head()

,Ahmedabad,Bangalore,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,field
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [86]:
#split the data to training and test
X_train,X_test,Y_train,Y_test=train_test_split( X, Output_df, test_size=0.2, random_state=0)

In [87]:
classification_model.fit(X_train,Y_train)

C:\AnacondaNew\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\AnacondaNew\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\AnacondaNew\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [88]:
predictions = classification_model.predict(X_test)

In [89]:
from sklearn.metrics import accuracy_score

In [90]:
accuracy_score(Y_test,predictions)

0.5396825396825397

In [ ]:
##Accuracy has improved with logistic regression

In [91]:
predictions

array(['Rajasthan Royals', 'Kings XI Punjab', 'Rajasthan Royals',
       'Kings XI Punjab', 'Rajasthan Royals', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Mumbai Indians',
       'Royal Challengers Bangalore', 'Royal Challengers Bangalore',
       'Rajasthan Royals', 'Delhi Daredevils', 'Deccan Chargers',
       'Mumbai Indians', 'Royal Challengers Bangalore', 'Mumbai Indians',
       'Rajasthan Royals', 'Delhi Daredevils', 'Delhi Daredevils',
       'Kolkata Knight Riders', 'Rajasthan Royals', 'Mumbai Indians',
       'Royal Challengers Bangalore', 'Rajasthan Royals',
       'Kolkata Knight Riders', 'Kolkata Knight Riders', 'Mumbai Indians',
       'Kolkata Knight Riders', 'Delhi Daredevils', 'Delhi Daredevils',
       'Mumbai Indians', 'Delhi Daredevils', 'Deccan Chargers',
       'Sunrisers Hyderabad', 'Delhi Daredevils', 'Chennai Super Kings',
       'Royal Challengers Bangalore', 'Mumbai Indians', 'Mumbai Indians',
       'Royal Challengers Bangalore', 'Rajasthan Royals'

In [65]:
Y_test

,winner
562,Chennai Super Kings
23,Mumbai Indians
41,Kings XI Punjab
609,Sunrisers Hyderabad
372,Deccan Chargers
65,Delhi Daredevils
408,Kings XI Punjab
88,Rajasthan Royals
556,Royal Challengers Bangalore
172,Royal Challengers Bangalore


In [67]:
X_test.head()

,Ahmedabad,Bangalore,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,field
562,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
41,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
609,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
372,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [68]:
X.head()

,Ahmedabad,Bangalore,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,Deccan Chargers,Delhi Daredevils,Gujarat Lions,Kings XI Punjab,Kochi Tuskers Kerala,Kolkata Knight Riders,Mumbai Indians,Pune Warriors,Rajasthan Royals,Rising Pune Supergiant,Rising Pune Supergiants,Royal Challengers Bangalore,Sunrisers Hyderabad,field
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
